In [73]:
%load_ext autoreload
%autoreload 2

import sys; sys.path.insert(0, '../../..') # add parent folder path where lib folder is

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:

import os

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from ml.models import common
from utils import helper, config, rayer, kaggle_dataset_helper
from ml.xai.non_model import simulate_knockoffs, KnockoffSetting


In [72]:
rayer.get_global_cluster()
# rayer.get_local_cluster()

In [54]:
def use_tokamat_ds():
    df = helper.get_tokamat_dataset()
    df = df.reset_index()

    df = common.label_encode(df)
    df = df.fillna(-1)

    potential_targets = ['WTOT', 'WTH', 'PLTH']
    df_y = df[potential_targets[0]]

    df_X = df[df.columns[~df.columns.isin(potential_targets)]]
    df_X = df_X.drop(['TOK_ID', 'LCUPDATE', 'DATE', 'NEL', 'ENBI'], axis = 1)

    
    return df_X, df_y
    # return train_test_split(X, y, test_size=0.33)


def use_covid_ds():
    df_X, df_y = helper.get_covid_dataset()
    df_X = df_X.drop(['location'], axis = 1)

    return df_X, df_y
    # return train_test_split(df_X, df_y, test_size=0.33)


def use_transaction_predictions_ds():
    ds_train, ds_test = kaggle_dataset_helper.get_transaction_predictions_dataset()
    ds_train = common.label_encode(ds_train)
    ds_test = common.label_encode(ds_test)

    ds_train = ds_train.fillna(-1)
    ds_test = ds_test.fillna(-1)

    df_X = ds_train.loc[:, ds_train.columns != 'target']
    df_y = ds_train['target']

    return df_X, df_y
    



def use_house_pricing_ds():
    ds_train, ds_test = kaggle_dataset_helper.get_house_prices_dataset()
    ds_train = common.label_encode(ds_train)
    ds_test = common.label_encode(ds_test)

    ds_train = ds_train.fillna(-1)
    ds_test = ds_test.fillna(-1)


    df_X = ds_train.loc[:, ds_train.columns != 'SalePrice']
    df_y = ds_train['SalePrice']

    return df_X, df_y

    


In [82]:
df_X, df_y = use_transaction_predictions_ds()

In [83]:
np.random.seed(KnockoffSetting.SEED)

X = df_X
y = df_y

fdr = 0.1
fstats = ['lasso', 'ridge', 'randomforest']

df = simulate_knockoffs(fdr=fdr, fstats=fstats, itr=2, df_X=X, df_y=y)


(__worker__ pid=851875) itr: 2, ksampler: 'gaussian',  fstat:ridge
(__worker__ pid=851936) itr: 1, ksampler: 'gaussian',  fstat:lasso
(__worker__ pid=851930) itr: 1, ksampler: 'gaussian',  fstat:ridge
(__worker__ pid=851933) itr: 2, ksampler: 'gaussian',  fstat:lasso
(__worker__ pid=851935) itr: 1, ksampler: 'gaussian',  fstat:randomforest
(__worker__ pid=851934) itr: 2, ksampler: 'gaussian',  fstat:randomforest


(__worker__ pid=851875) /home/rayuser/.local/lib/python3.8/site-packages/knockpy/mrc.py:276: UserWarning: 
(__worker__ pid=851875) 	---------------------------------------------------------
(__worker__ pid=851875) 	---------------------------------------------------------
(__worker__ pid=851875) 	The choldate package is not installed:
(__worker__ pid=851875) 	solving for MVR or maxent knockoffs without it will be 
(__worker__ pid=851875) 	~5x slower. For instructions on how to install choldate,
(__worker__ pid=851875) 	see https://amspector100.github.io/knockpy/installation.html.
(__worker__ pid=851875) 	To disable this warning, set choldate_warning=False as an 
(__worker__ pid=851875) 	argument or a knockoff_kwarg.
(__worker__ pid=851875) 	----------------------------------------------------------
(__worker__ pid=851875) 	----------------------------------------------------------
(__worker__ pid=851875) 
(__worker__ pid=851875)   warnings.warn(constants.CHOLDATE_WARNING)
(__worker__ p

In [85]:
df

cols
var_68     201.0
var_27     200.0
var_183    199.0
var_50     198.0
var_38     197.0
           ...  
var_73       0.0
var_74       0.0
var_75       0.0
var_76       0.0
var_199      0.0
Length: 201, dtype: float64

Log channel is reconnecting. Logs produced while the connection was down can be found on the head node of the cluster in `ray_client_server_[port].out`
2022-12-07 23:46:49,707	WARNING dataclient.py:396 -- Encountered connection issues in the data channel. Attempting to reconnect.
2022-12-07 23:47:19,922	WARNING dataclient.py:403 -- Failed to reconnect the data channel
